In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

from credit_g_dataset import get_preprocessed_credit_g_dataset

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 6)

# Load version 1 of the dataset credit-g

In [2]:
%%capture
X_train, X_validation, X_test, y_train, y_validation, y_test = get_preprocessed_credit_g_dataset()

## Train the model using the training set and adjust hyperparameters with Optuna using the validation set

In [3]:
random_state=0

In [4]:
def objective(trial):
    params = {
    "solver": trial.suggest_categorical("solver", ["lbfgs", "liblinear", "sag", "saga"]),        
    "max_iter": 300,
    "n_jobs": -1,
    "random_state": random_state
    }
    
    model = LogisticRegression(**params).fit(X_train, y_train)

    y_predicted = model.predict(X_validation)   
    (tn, fp, fn, tp) = confusion_matrix(y_validation, y_predicted).ravel()
    f1_val = (2*tp)/(2*tp+fp+fn)
    
    return f1_val


In [5]:
start_time = time.time()

study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials=100)

total_time = time.time()-start_time
print(f"total_time={total_time}")
print(study.best_params)
print(study.best_value)

[I 2023-12-20 18:17:20,608] A new study created in memory with name: no-name-7b834236-2738-493d-8f38-3bd8069d6794
C:\Users\DLP\miniconda3\envs\elpi\Lib\site-packages\sklearn\linear_model\_logistic.py:1223: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 16.
  warnings.warn(
[I 2023-12-20 18:17:20,620] Trial 0 finished with value: 0.8585858585858586 and parameters: {'solver': 'liblinear'}. Best is trial 0 with value: 0.8585858585858586.
[I 2023-12-20 18:17:22,188] Trial 1 finished with value: 0.8585858585858586 and parameters: {'solver': 'lbfgs'}. Best is trial 0 with value: 0.8585858585858586.
C:\Users\DLP\miniconda3\envs\elpi\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2023-12-20 18:17:22,307] Trial 2 finished with value: 0.8585858585858586 and parameters: {'solver': 'saga'}. Best is trial 0 with value: 0.858585858585858

total_time=17.190016984939575
{'solver': 'liblinear'}
0.8585858585858586


In [6]:
best_solver = study.best_params["solver"]

In [7]:
model = LogisticRegression(solver=best_solver, max_iter=300, n_jobs=-1, random_state=random_state)

## Run on the test set and use training and validation sets for training

In [8]:
X_train_valid = np.concatenate((X_train, X_validation))
y_train_valid = np.concatenate((y_train, y_validation))

model.fit(X_train_valid, y_train_valid)
print(model.score(X_test, y_test))
y_predicted = model.predict(X_test)

(tn, fp, fn, tp) = confusion_matrix(y_test, y_predicted).ravel()
precision_val = float(tp)/float(tp+fp)
recall_val = float(tp)/float(tp+fn)
specificity_val = float(tn)/float(tn+fp)
accuracy_val = float(tp+tn)/float(tn+fp+fn+tp)
f1_val = (2*tp)/(2*tp+fp+fn)
mcc_val = float((tp*tn)-(fp*fn))/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))


print("\n")
print("Test set precision_val =", precision_val)
print("Test set recall_val =", recall_val)
print("Test set sspecificity =", specificity_val)
print("Test set mcc_val =", mcc_val)
print("Test set accuracy_val =", accuracy_val)
print("Test set f1_val =", f1_val)

0.73


Test set precision_val = 0.8333333333333334
Test set recall_val = 0.8
Test set sspecificity = 0.52
Test set mcc_val = 0.3086066999241838
Test set accuracy_val = 0.73
Test set f1_val = 0.8163265306122449


C:\Users\DLP\miniconda3\envs\elpi\Lib\site-packages\sklearn\linear_model\_logistic.py:1223: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 16.
  warnings.warn(


In [9]:
confusion_matrix(y_test, y_predicted)

array([[13, 12],
       [15, 60]], dtype=int64)

In [10]:
y_test

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1])